In [1]:
from PIL import Image, ImageFont, ImageDraw
from pilmoji import Pilmoji
from pilmoji.source import EmojiCDNSource
import numpy as np
import pickle

In [2]:
def create_data(emojis,
                size=50,
                save_as=None, 
                disp=True):
    """
    Create an dict containing all data to draw N 
    different emojis (which are specified in the
    first parameter of function). Structure of dict: 
    {
        'emojis': list of N emojis of str type,
        'styles': dict of 4 style names {number: stylename},
        'size': int (side of square image on which emojis are drawn),
        'matrices': np-array with shape (4, N, size, size, 4) of uint8,
    }

    PARAMETERS
    ----------
    emojis : list[str]
        list of emojis to build data

    size : int
        length of the side of the square image, on
        which emojis will be drawn

    save_as : str or None
        path to save data. If None is specified, data
        will not be saved
    
    disp : bool
        toggle progress displaing
    """
    
    def _str2style(name):
        """to convert style name to style class"""
        class StyleClass(EmojiCDNSource):
            STYLE = name
        return StyleClass
        
    # Quantity of emojis
    n_emojis = len(emojis)

    # Size of image with emoji
    esize = (size, size)
    
    data_dict = {
        'emojis': emojis,
        'styles': {0: 'twitter', 
                   1: 'apple', 
                   2: 'google', 
                   3: 'facebook'},
        'size': size,
        'matrices': np.zeros((4, n_emojis, *esize, 4), 
                             dtype='uint8')
    }

    # Style classes
    styles = [_str2style(data_dict['styles'][i]) for i in range(4)]

    for i, emoji in enumerate(emojis):
        for j, style in enumerate(styles):
            # Drawing an emoji on empty (0, 0, 0, 0) RGBA image
            with Image.new('RGBA', esize, 0) as image:
                font = ImageFont.truetype('arial.ttf', esize[0])
                with Pilmoji(image, source=style) as pilmoji:
                    pilmoji.text((0, 0), emoji, font=font)
                    
            # RGBA-matrix
            image_np = np.array(image)
            data_dict['matrices'][j, i] = image_np
            
            # Displaing the progress
            if disp:
                total = n_emojis * 4
                current = i * 4 + j + 1
                progress = f'{round(current / total * 100, 2)}%'
                print(f'\r{progress:<6} ({i+1}/{len(emojis)})', end='')
    if disp:
        print()

    # Saving as binary pickle file
    if save_as is not None:
        with open(f'{save_as}.pkl', 'wb') as file:
            pickle.dump(data_dict, file)

    return data_dict

# <font color="red">↓ TO DELETE ↓

In [3]:
def load_emoji_list(list_name):
    with open(f'lists/{list_name}.txt', 'r') as file:
        emoji_list = [line.strip() for line in file]
    return emoji_list

In [6]:
def load_data(pack):
    with open(f'{pack}.pkl', 'rb') as file:
        data = pickle.load(file)
    return data